<a href="https://colab.research.google.com/github/jacobdwatters/NIOSH-Project/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install torchmetrics

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch import nn, tensor
from torch.optim import SGD
from torch.utils.data import DataLoader, Dataset
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

from torchmetrics import R2Score

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 32.4 MB/s 


In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [40]:
violation_data = pd.read_csv('/content/gdrive/My Drive/NIOSH Project/data/violations_processed.csv', index_col=0)

In [41]:
FEATURES = ['MINE_TYPE', 'COAL_METAL_IND', 'SIG_SUB', 'LIKELIHOOD', 
            'INJ_ILLNESS', 'NO_AFFECTED', 'NEGLIGENCE', 'VIOLATOR_VIOLATION_CNT',
            'VIOLATOR_INSPECTION_DAY_CNT']
TARGETS = ['PROPOSED_PENALTY']

X = violation_data[FEATURES]
y = violation_data[TARGETS]

In [42]:
# Define which columns should be encoded vs scaled
columns_to_encode = ['MINE_TYPE', 'COAL_METAL_IND', 'LIKELIHOOD', 'INJ_ILLNESS', 'SIG_SUB', 'NEGLIGENCE']
columns_to_scale  = ['VIOLATOR_VIOLATION_CNT', 'NO_AFFECTED', 'VIOLATOR_INSPECTION_DAY_CNT']

# Instantiate encoder/scaler
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False)

target_scaler = StandardScaler()

# Scale and Encode Separate Columns
scaled_columns  = scaler.fit_transform(X[columns_to_scale])
encoded_columns = ohe.fit_transform(X[columns_to_encode])

# Concatenate (Column-Bind) Processed Columns Back Together
X_pre = np.concatenate([scaled_columns, encoded_columns], axis=1)
np.nan_to_num(X_pre, copy=False)

y_pre = target_scaler.fit_transform(y)

print('Features shape:', X_pre.shape)

Features shape: (2755922, 24)


In [102]:
X_train, X_test, y_train, y_test = train_test_split(X_pre, y_pre, test_size = 0.25, random_state = 0)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

class ViolationDataset(Dataset):
    def __init__(self, features, targets):
        self.features = tensor(features)
        self.targets = tensor(targets)
    
    def __len__(self):
        return len(self.targets)
      
    def __getitem__(self, index):
        return self.features[index], self.targets[index]

train_dataset = ViolationDataset(X_train, y_train)
test_dataset = ViolationDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

(tensor([0.9049, 0.5556, 0.3927, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000]), tensor([0.6788]))

In [96]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [103]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(24, 48),
            nn.ReLU(),
            nn.Linear(48, 48),
            nn.ReLU(),
            nn.Linear(48, 24),
            nn.ReLU(),
            nn.Linear(24, 1)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [104]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=24, out_features=48, bias=True)
    (1): ReLU()
    (2): Linear(in_features=48, out_features=48, bias=True)
    (3): ReLU()
    (4): Linear(in_features=48, out_features=24, bias=True)
    (5): ReLU()
    (6): Linear(in_features=24, out_features=1, bias=True)
  )
)


In [105]:
criterion = nn.MSELoss()
optimizer = SGD(model.parameters(), lr=0.0001, momentum=0.9)
try:
    model.train()
    for epoch in range(2):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
except KeyboardInterrupt:
    pass

model.eval()
print('Finished Training')

[1,  2000] loss: 1.234
[1,  4000] loss: 0.933
[1,  6000] loss: 0.896
[1,  8000] loss: 0.772
[1, 10000] loss: 0.825
[1, 12000] loss: 0.764
[1, 14000] loss: 1.208
[1, 16000] loss: 1.032
[1, 18000] loss: 0.865
[1, 20000] loss: 1.306
[1, 22000] loss: 0.934
[1, 24000] loss: 0.798
[1, 26000] loss: 1.157
[1, 28000] loss: 0.654
[1, 30000] loss: 1.352
[1, 32000] loss: 0.706
[1, 34000] loss: 0.937
[1, 36000] loss: 0.813
[1, 38000] loss: 0.983
[1, 40000] loss: 0.784
[1, 42000] loss: 0.760
[1, 44000] loss: 1.226
[1, 46000] loss: 0.528
[1, 48000] loss: 0.650
[1, 50000] loss: 0.480
[1, 52000] loss: 0.471
[1, 54000] loss: 0.751
[1, 56000] loss: 0.486
[1, 58000] loss: 1.333
[1, 60000] loss: 0.642
[1, 62000] loss: 0.905
[1, 64000] loss: 0.788
[1, 66000] loss: 0.757
[1, 68000] loss: 0.656
[1, 70000] loss: 0.885
[1, 72000] loss: 0.638
[1, 74000] loss: 0.612
[1, 76000] loss: 0.659
[1, 78000] loss: 0.627
[1, 80000] loss: 0.588
[1, 82000] loss: 0.386
[1, 84000] loss: 0.645
[1, 86000] loss: 0.462
[1, 88000] 

In [106]:
y_pred = model(tensor(X_test))
R2Score()(y_pred, tensor(y_test))

tensor(0.4278, grad_fn=<SqueezeBackward0>)